In [1]:
import pandas as pd
import tinvest as ti
from tinvest import CandleResolution
from tinvest import TooManyRequestsError, UnexpectedError

from threading import Thread
from tqdm.notebook import tqdm

import os
import datetime as dt
import time

import sqlite3
import investpy
import yfinance as yf

In [2]:
pd.options.display.max_columns = 150

### Источники данных
Рассматривалось использование трех основных источников информации об акциях в качестве возможных: 
- tinvest, SDK для Tinkoff Инвестиции API
- investpy, библиотека для использования данных с сайта investing.com
- yfinanse, библиотека для и использования yahoo finanse API

#### API Tinkoff Инвестиции
Акуальная документация соответсвует версии библиотек 3.0 и выше, для которых необходимо использование python версией выше 3.8, поэтому для запуска кода, использующего эту библиотеку необходимо использование указанной версии python.

Для использования API необходимо наличие токена, который можно завести в личном кабинете Тинькофф. Ограничения API - 240 операций в секунду на запросы по данным рынка. Для работы с API использовалось неофициальное SDK - tinvest - упомянутое на официальной странице API.

Данный источник является основным по историческим данным ежедневных и ежечасовых продаж за последний год. Для устранения необходимости сохранения данных каждый раз используется база данных, взаимодействие с которой осуществляется с помощью sqlite3. Также, в качестве примера данных, в папке лежат данные в файлах tinvest_daily_data.csv и tinvest_stocks_info.xlsx.

In [26]:
TOKEN = os.getenv('TINVEST_TOKEN')

In [27]:
# client = ti.SyncClient(TOKEN)
# response = client.get_market_stocks()
# result = response.dict()["payload"]["instruments"]
# candle = client.get_market_candles(
#     "BBG000B9X8C0",
#     "2020-06-01T00:00:00+03:00",
#     "2020-06-30T00:00:00+03:00",
#     CandleResolution("day"),
# ).dict()["payload"]["candles"]

# ti_fundamental_df = pd.DataFrame(result)
# candle_df = pd.DataFrame(candle)
# display(ti_fundamental_df.head())
# display(candle_df.head())

,currency,figi,isin,lot,min_price_increment,name,ticker,type,min_quantity
0,Currency.usd,BBG000HLJ7M4,US45867G1013,1,0.01,InterDigItal Inc,IDCC,InstrumentType.stock,NaN
1,Currency.usd,BBG000BDSLD7,US8030542042,1,0.01,АДР SAP,SAP,InstrumentType.stock,NaN
2,Currency.usd,BBG002293PJ4,US74967X1037,1,0.01,RH,RH,InstrumentType.stock,NaN
3,Currency.usd,BBG000CTQBF3,US8552441094,1,0.01,Starbucks Corporation,SBUX,InstrumentType.stock,NaN
4,Currency.usd,BBG000BH0FR6,US81181C1045,1,0.01,Seagen Inc.,SGEN,InstrumentType.stock,NaN


,c,figi,h,interval,l,o,time,v
0,74.37,BBG000B9X8C0,75.68,CandleResolution.day,73.86,75.14,2020-06-01 07:00:00+00:00,293707
1,74.91,BBG000B9X8C0,75.25,CandleResolution.day,74,74.38,2020-06-02 07:00:00+00:00,283474
2,76.08,BBG000B9X8C0,76.21,CandleResolution.day,75.08,75.21,2020-06-03 07:00:00+00:00,408469
3,73.99,BBG000B9X8C0,76.5,CandleResolution.day,73.1,75.66,2020-06-04 07:00:00+00:00,282975
4,75.58,BBG000B9X8C0,76.96,CandleResolution.day,74.44,74.96,2020-06-05 07:00:00+00:00,411271


Заполнение БД осуществляется с помощью скриптов в папке db. Из-за ограничения кол-ва запросов их проверять дольше чем остальное, поэтому рекомендую сразу использовать БД при проверке. 

In [5]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

In [6]:
database = r"pythonsqlite.db"
conn = create_connection(database)

2.6.0


In [7]:
stocks = pd.read_sql(sql='Select * from stocks', con=conn)

In [10]:
stocks.head()

,UPDATE_DATE,CURRENCY,FIGI,ISIN,LOT,MIN_PRICE_INCREMENT,NAME,TICKER,TYPE,MIN_QUANTITY
0,2021-06-10 15:18:40.216564,USD,BBG000HLJ7M4,US45867G1013,1,0.01,InterDigItal Inc,IDCC,Stock,NaN
1,2021-06-10 15:18:40.223985,USD,BBG000BDSLD7,US8030542042,1,0.01,АДР SAP,SAP,Stock,NaN
2,2021-06-10 15:18:40.231109,USD,BBG002293PJ4,US74967X1037,1,0.01,RH,RH,Stock,NaN
3,2021-06-10 15:18:40.238816,USD,BBG000CTQBF3,US8552441094,1,0.01,Starbucks Corporation,SBUX,Stock,NaN
4,2021-06-10 15:18:40.245532,USD,BBG000BPL8G3,US5537771033,1,0.01,MTS Systems Corp,MTSC,Stock,NaN


#### Investing.com
У сайта есть свой интерфейс по выгрузке данных, с которым можно ознакомится, используя библиотеку investpy. Рассмотрим доступные данные по нескольким бумагам

In [16]:
i_com_fundamentals_df = investpy.get_stocks()

sample = stocks.sample(5)

Рынок бумаг Тинькоф значительно меньше чем глобальный рынок, что приносит свои проблемы в интерпретации данных, которые можно получить от investing.com. По одному тикеру можно получить сразу несколько позиций, связанных с разными странами, а то и разными компаниями. 

Скорей всего тут есть своя, пока не известная мне специфика, поэтому на текущий момент будут рассматриваться только данные, связанные с бумагами доступными через инструменты Тинькофф.

In [25]:
print('Ценных бумаг в доступе Тинькофф Инвестиции - {0} штук'.format(len(stocks)))
print('Ценных бумаг в базе Investing.com - {0} штук'.format(len(i_com_fundamentals_df)))

Ценных бумаг в доступе Тинькофф Инвестиции - 1695 штук
Ценных бумаг в базе Investing.com - 39952 штук


In [21]:
i_com_fundamentals_df.merge(sample, how = 'inner', left_on = ['currency', 'symbol'], right_on = ['CURRENCY', 'TICKER'])

,country,name,full_name,isin,currency,symbol,UPDATE_DATE,CURRENCY,FIGI,ISIN,LOT,MIN_PRICE_INCREMENT,NAME,TICKER,TYPE,MIN_QUANTITY
0,chile,Chevron,Chevron Corp,US1667641005,USD,CVX,2021-06-10 15:18:49.270267,USD,BBG000K4ND22,US1667641005,1,0.0100,Chevron,CVX,Stock,NaN
1,united states,Chevron,Chevron Corp,US1667641005,USD,CVX,2021-06-10 15:18:49.270267,USD,BBG000K4ND22,US1667641005,1,0.0100,Chevron,CVX,Stock,NaN
2,belgium,Chevron,Chevron Corporation,US1667641005,USD,CVX,2021-06-10 15:18:49.270267,USD,BBG000K4ND22,US1667641005,1,0.0100,Chevron,CVX,Stock,NaN
3,united states,Citigroup,Citigroup Inc,US1729674242,USD,C,2021-06-10 15:18:53.711548,USD,BBG000FY4S11,US1729674242,1,0.0100,Citigroup,C,Stock,NaN
4,united states,InterDigital,InterDigital Inc,US45867G1013,USD,IDCC,2021-06-10 15:18:40.216564,USD,BBG000HLJ7M4,US45867G1013,1,0.0100,InterDigItal Inc,IDCC,Stock,NaN
5,united states,Fortive,Fortive Corp,US34959J1088,USD,FTV,2021-06-10 15:18:45.509865,USD,BBG00BLVZ228,US34959J1088,1,0.0100,Fortive,FTV,Stock,NaN
6,russia,MosEnrg,MosEnrg,RU0008958863,RUB,MSNG,2021-06-10 15:18:47.197084,RUB,BBG004S687W8,RU0008958863,1000,0.0005,Мосэнерго,MSNG,Stock,NaN


In [19]:
i_com_fundamentals_df[i_com_fundamentals_df['symbol'].isin(sample['TICKER'])]

,country,name,full_name,isin,currency,symbol
62,argentina,Chevron DRC,Chevron Corp DRC,ARDEUT110087,ARS,CVX
63,argentina,Citigroup DRC,Citigroup Inc DRC,ARDEUT110426,ARS,C
1983,canada,Cematrix Corporation,Cematrix Corporation,CA15120R1038,CAD,CVX
2424,chile,Chevron,Chevron Corp,US1667641005,USD,CVX
2577,mexico,Citigroup,Citigroup Inc.,US1729674242,MXN,C
2724,mexico,Chevron,Chevron Corporation,US1667641005,MXN,CVX
3159,mexico,Chevron,Chevron Corp,US1667641005,MXN,CVX
3259,united states,Chevron,Chevron Corp,US1667641005,USD,CVX
3260,united states,Citigroup,Citigroup Inc,US1729674242,USD,C
4476,united states,InterDigital,InterDigital Inc,US45867G1013,USD,IDCC


In [ ]:
Через данный сервис также можно получать информацию об исторических продажах, что однако ограничено дневными, недельными и месячными срезами, в отличии от двух других источников.

In [33]:
appl_daily_df = investpy.get_stock_historical_data(stock='AAPL',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2020', interval='Daily')

In [35]:
appl_daily_df.head()

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2010-01-04,7.62,7.66,7.58,7.64,493728704,USD
2010-01-05,7.66,7.70,7.62,7.66,601904640,USD
2010-01-06,7.66,7.69,7.53,7.53,552158912,USD
2010-01-07,7.56,7.57,7.47,7.52,477129760,USD
2010-01-08,7.51,7.57,7.47,7.57,447876768,USD


#### Yahoo Finance API
Позволяет получить информацию используя данные сайта Yahoo Finance. Существуют как платные, так и бесплатные инструментарии для работы с эти API. Для ограниченной выгрузки данных можно использовать бесплатный, одним из которых является библиотека yfinance

In [36]:
aapl = yf.Ticker('aapl')

In [39]:
aapl.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 100000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [38]:
aapl.history(
    interval='1h', 
    start="2021-06-02", 
    end= "2021-06-03"
)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2021-06-02 09:30:00-04:00,124.279999,125.190002,124.110001,125.084999,16148036,0,0
2021-06-02 10:30:00-04:00,125.079697,125.239998,124.650002,124.970001,8434366,0,0
2021-06-02 11:30:00-04:00,124.970001,125.180000,124.620003,124.706802,5315912,0,0
2021-06-02 12:30:00-04:00,124.705002,124.709999,124.070000,124.195000,5513431,0,0
2021-06-02 13:30:00-04:00,124.199997,124.510002,124.050003,124.480003,5447741,0,0
2021-06-02 14:30:00-04:00,124.485001,124.794998,124.410004,124.785004,5563554,0,0
2021-06-02 15:30:00-04:00,124.790001,125.168999,124.660004,125.089996,6289304,0,0
2021-06-18 10:54:59-04:00,130.679993,130.679993,130.679993,130.679993,0,0,0
